### Git versioning

In [7]:
import git  #Requires Gitpython installed, to install do "$ pip install Gitpython" in your terminal
import pkg_resources
openfisca_france_location = pkg_resources.get_distribution('openfisca-france').location
repo = git.Repo(openfisca_france_location)
repo.git.status()

sha = repo.head.object.hexsha

print "---This notebook have been runed with--- \n","Openfisca-france :", repo.head.object.hexsha, '\n' 
print "Commit short SHA:", repo.git.rev_parse(sha, short=4)
print "Branch: ", repo.git.rev_parse('--abbrev-ref', "HEAD") #equivalent to: $git rev-parse --abbrev-ref HEAD
print "Date (y/m/d):", repo.git.show("-s", '--format=%ci', 'HEAD^')[:-5],"\n"  # equivalent to: $git show -s --format=%ci HEAD^

print "Openfisca-core :", git.Repo(pkg_resources.get_distribution('openfisca-core').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('openfisca-core').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "Openfisca-france-data :", git.Repo(pkg_resources.get_distribution('openfisca-france-data').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('openfisca-france-data').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "OpenFisca-Aggregates :", git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]
print "OpenFisca-Survey-Manager :", git.Repo(pkg_resources.get_distribution('OpenFisca-Survey-Manager').location).head.object.hexsha, ' Date:', git.Repo(pkg_resources.get_distribution('OpenFisca-Aggregates').location).git.show("-s", '--format=%ci', 'HEAD^')[:-5]

---This notebook have been runed with--- 
Openfisca-france : 03f19d8d1c056fec5562f592814f9526b3e6c0a2 

Commit short SHA: 03f19
Branch:  mensualisation_wip
Date (y/m/d): 2016-06-01 08:56:21  

Openfisca-core : 4076f591f4bcb571391b65a989e1603fdc4cb759  Date: 2016-02-24 16:11:57 
Openfisca-france-data : 4581a82fd1456cbe0bdd86ab1b57db6fef37c8cf  Date: 2016-05-31 15:46:50 
OpenFisca-Aggregates : fd210d8cfdf6ed68f173d9d5c3eab4a4302ed2b9  Date: 2016-04-04 11:35:03 
OpenFisca-Survey-Manager : 868512d24307aa75cfa7bbbf8420542960963966  Date: 2016-04-04 11:35:03 


In [1]:
from openfisca_core import base_functions

def requested_period_default_value_on_family(formula, simulation, period, *extra_params):
    if formula.function is not None:
        return formula.function(simulation, period, *extra_params)
    holder = formula.holder
    import ipdb; ipdb.set_trace()
    column = holder.column
    array = np.empty(holder.entity.count, dtype = column.dtype)
    array.fill(column.default)
    return period, array

 
base_functions.requested_period_default_value = requested_period_default_value_on_family 

In [2]:
#del base_functions

### Imports

In [3]:
%matplotlib inline
from __future__ import division

from openfisca_core import periods
from openfisca_plugin_aggregates.tests.reform.test_aggregates_mensualized import create_survey_scenario
import pandas as pd

import matplotlib.pyplot as plt

import numpy as np
from numpy import (datetime64, logical_and as and_, logical_not as not_, logical_or as or_, logical_xor as xor_,
maximum as max_, minimum as min_, round)


#Faire des filtres pour Mahdi survey_scenario.create_data_frame_by_entity_key_plural

pd.set_option('expand_frame_repr', False)  #utile quand on utilise pandas en mode console
pd.set_option("display.max_rows",201)
pd.set_option("display.max_columns",201)

# Create Survey Scenario

In [4]:
period = periods.period(2009)

In [5]:
period.this_year.this_month

Period((u'month', Instant((2009, 1, 1)), 1))

In [6]:
survey_scenario = create_survey_scenario(2009)
survey_scenario.new_simulation()

In [9]:

reload(base_functions)
base_functions.requested_period_default_value = base_functions.requested_period_default_value

In [10]:
#%pdb
survey_scenario.simulation.calculate_add("revdisp_mensuel_compensated_ir_mensuel")

ERROR:openfisca_core.formulas:An error occurred while calling formula revdisp_mensuel_compensated_ir_mensuel@menages<2009> in module openfisca_france.reforms.mensualisation.mensualisation_ir_8


AssertionError: Circular definition detected on formula revdisp_mensuel_compensated_ir_mensuel<2009>. Formulas and periods involved: chomage_brut<2009-01>, csg_imposable_chomage<2009-01>, pen_mensuel<2009-01>, revdisp_mensuel_compensated_ir_mensuel<2009>.

In [ ]:
formula.function(simulation, period, extra_params)



In [ ]:
print survey_scenario.simulation.persons.simulation.entity_by_key_plural

In [ ]:
simulation.entity_by_key_singular['menage'].count

In [ ]:
from openfisca_core import holders


def sum_by_entity( array_or_dated_holder, entity, roles = None, simulation = simulation):
    #1# Sert juste à récupérer la simulation.
    simulation
    persons = simulation.persons


    #2#
    assert entity in simulation.entity_by_key_singular, u"Unknown entity: {}".format(entity).encode('utf-8')
    entity = simulation.entity_by_key_singular[entity]
    assert not entity.is_persons_entity


    #3#
    if isinstance(array_or_dated_holder, (holders.DatedHolder, holders.Holder)):
        assert array_or_dated_holder.entity.is_persons_entity
        array = array_or_dated_holder.array
    else:
        array = array_or_dated_holder
        assert isinstance(array, np.ndarray), u"Expected a holder or a Numpy array. Got: {}".format(array).encode(
            'utf-8')
        assert array.size == persons.count, u"Expected an array of size {}. Got: {}".format(persons.count,
            array.size)

    entity_index_array = persons.simulation.holder_by_name[entity.index_for_person_variable_name].array
    if roles is None:
        roles = range(entity.roles_count)
    target_array = np.zeros( entity.count ,dtype = array.dtype if array.dtype != np.bool else np.int16)
    for role in roles:
        # TODO: Mettre les filtres en cache dans la simulation
        boolean_filter = persons.simulation.holder_by_name[entity.role_for_person_variable_name].array == role
        target_array[entity_index_array[boolean_filter]] += array[boolean_filter]
    return target_array

In [ ]:
irpp = simulation.calculate('irpp')

sum_by_entity(weight_individus, 'foyer_fiscal')

In [ ]:
print simulation.calculate('idfam').shape
print simulation.calculate('idfam').shape
print simulation.calculate('idfam').shape

In [ ]:
#simulation.calculate('idfam')
pd.Series(simulation.calculate('champm_individus')).value_counts()

In [ ]:
#simulation.calculate('idfam')
pd.Series(simulation.calculate('champm_foyers_fiscaux')).value_counts()

In [ ]:

def any_by_roles(array_or_dated_holder_input, array_or_dated_holder_target, entity = None, roles = None):
    holder = array_or_dated_holder_input.holder
    target_entity = holder.entity
    simulation = target_entity.simulation
    persons = simulation.persons
    if entity is None:
        entity = holder.entity
    else:
        assert entity in simulation.entity_by_key_singular, u"Unknown entity: {}".format(entity).encode('utf-8')
        entity = simulation.entity_by_key_singular[entity]
    assert not entity.is_persons_entity
    if isinstance(array_or_dated_holder_target, (holders.DatedHolder, holders.Holder)):
        assert array_or_dated_holder_target.entity.is_persons_entity
        array = array_or_dated_holder_target.array
    else:
        array = array_or_dated_holder_target
        assert isinstance(array, np.ndarray), u"Expected a holder or a Numpy array. Got: {}".format(array).encode(
            'utf-8')
        assert array.size == persons.count, u"Expected an array of size {}. Got: {}".format(persons.count,
            array.size)
    entity_index_array = persons.simulation.holder_by_name[entity.index_for_person_variable_name].array
    if roles is None:
        roles = range(entity.roles_count)
    target_array = self.zeros(dtype = np.bool)
    for role in roles:
        # TODO Mettre les filtres en cache dans la simulation
        boolean_filter = persons.simulation.holder_by_name[entity.role_for_person_variable_name].array == role
        target_array[entity_index_array[boolean_filter]] += array[boolean_filter]
    return target_array

In [ ]:
irpp = simulation.compute('irpp')

any_by_roles(irpp, weight_menages.values)

In [ ]:
import ipdb; ipdb.set_trace()
survey_scenario.simulation.calculate("noindiv")

In [ ]:


def function_creator(postes_coicop, year_start = None, year_stop = None):
    start = date(year_start, 1, 1) if year_start is not None else None
    stop = date(year_stop, 12, 31) if year_stop is not None else None
    if len(postes_coicop) != 0:
        @dated_function(start = start, stop = stop)
        def func(self, simulation, period):
            return period, sum(simulation.calculate(
                'poste_' + slugify(poste, separator = u'_'), period) for poste in postes_coicop
                )
        func.__name__ = "function_{year_start}_{year_stop}".format(year_start = year_start, year_stop = year_stop)
        return func

    else:  # To deal with Reform emptying some fiscal categories
        @dated_function(start = start, stop = stop)
        def func(self, simulation, period):
            return period, self.zeros()

        func.__name__ = "function_{year_start}_{year_stop}".format(year_start = year_start, year_stop = year_stop)
        return func



def generate_variables(categories_fiscales = None, Reform = None, tax_benefit_system = None):
    assert categories_fiscales is not None
    original_categories = sorted(categories_fiscales_data_frame['categorie_fiscale'].drop_duplicates())
    removed_categories = set()
    if Reform:
        removed_categories = set(original_categories).difference(
            set(categories_fiscales['categorie_fiscale'].drop_duplicates()))

    for categorie_fiscale in original_categories:
        year_start = 1994
        year_final_stop = 2014
        functions_by_name = dict()
        for year in range(year_start, year_final_stop + 1):
            postes_coicop = sorted(
                categories_fiscales.query(
                    'start <= @year and stop >= @year and categorie_fiscale == @categorie_fiscale'
                    )['code_coicop'].astype(str))
            if year == year_start:
                previous_postes_coicop = postes_coicop
                continue

            if previous_postes_coicop == postes_coicop and year != year_final_stop:
                continue
            else:
                year_stop = year - 1 if year != year_final_stop else year_final_stop

                dated_func = function_creator(previous_postes_coicop, year_start = year_start, year_stop = year_stop)
                dated_function_name = u"function_{year_start}_{year_stop}".format(
                    year_start = year_start, year_stop = year_stop)
                log.info(u'Creating fiscal category {} ({}-{}) with the following products {}'.format(
                    categorie_fiscale, year_start, year_stop, postes_coicop))

                if len(previous_postes_coicop) != 0:
                    functions_by_name[dated_function_name] = dated_func

                if len(previous_postes_coicop) == 0 and categorie_fiscale in removed_categories:
                    functions_by_name[dated_function_name] = dated_func

                year_start = year

            previous_postes_coicop = postes_coicop

        class_name = u'depenses_{}'.format(categorie_fiscale)
        # Trick to create a class with a dynamic name.
        if not Reform:
            definitions_by_name = dict(
                column = FloatCol,
                entity_class = Menages,
                label = u"Categorie fiscale {0}".format(categorie_fiscale),
                )
            definitions_by_name.update(functions_by_name)
            type(class_name.encode('utf-8'), (DatedVariable,), definitions_by_name)
        else:
            definitions_by_name = dict(
                reference = tax_benefit_system.column_by_name[class_name.encode('utf-8')]
                )
            definitions_by_name.update(functions_by_name)
            type(class_name.encode('utf-8'), (Reform.DatedVariable,), definitions_by_name)

        del definitions_by_name

In [ ]:
generate_variables(categories_fiscales = None, Reform = None, tax_benefit_system = simulation.tax_benefit_system)

In [ ]:
simulation.tax_benefit_system

In [ ]:
sum_by_entity(weight_individus, 'individu')

In [ ]:
entity = simulation.entity_by_key_singular['famille']
entity.count

In [ ]:
survey_scenario.simulation.calculate("noindiv")

In [ ]:
entity.iter_member_persons_role_and_id()

In [ ]:
from openfisca_utils import make_ready_to_use_scenario

In [ ]:
scenario = make_ready_to_use_scenario.make_single_with_child_scenario(year = 2009)

In [ ]:
scenario.test_case


In [ ]:
c = np.zeros(1)
a = np.array([1,2,3])
b = np.array([True,True,False])
c += a[b]; c

In [ ]:
simulation = survey_scenario.simulation

In [ ]:
entity=  simulation.holder_by_name['aah'].entity
entity.key_singular

In [ ]:
coucou = survey_scenario.simulation.compute("retraite_imposable", "2009-{}".format(month))

In [ ]:
coucou.entity

In [ ]:
simulation.entity_by_key_singular